<a href="https://colab.research.google.com/github/vasiliyeskin/MachineLearningExperiences/blob/master/FashionMNIST/fashion_mnist_keras_tuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Оптимизация гиперпараметров нейросети с помощью [Keras Tuner](https://github.com/keras-team/keras-tuner)

Взято с https://www.youtube.com/watch?v=M08yHFPKTNI&list=PLtPJ9lKvJ4oiz9aaL_xcZd-x0qd8G0VN_&index=11.

Чтобы запускать и редактировать код, сохраните копию этого ноутбука себе (File->Save a copy in Drive...). Свою копию вы сможете изменять и запускать.

Не забудьте подключить GPU, чтобы сеть обучалась быстрее (Runtime -> Change Runtime Type -> Hardware Accelerator -> GPU).



## Гиперпараметры обучения нейронной сети

- Количество слоев нейронной сети
- Количество нейронов в каждом слое
- Функции активации, которые используются в слоях
- Тип оптимизатора при обучении нейронной сети
- Количество эпох обучения

## Установка Keras Tuner

In [1]:
pip install -U keras-tuner

     |████████████████████████████████| 61kB 1.8MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=c2cbd0e4907114b18b1b0dc07a8325416692e92b9cca2f6b2374ec228fe9f8a3
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=eb229c3fdde860218aa215eba19b7653e690b06e086b40eae80fc60ccf3d0bfc
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


## Подключаем нужные пакеты

In [2]:
%tensorflow_version 2.x
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import utils
from google.colab import files
from kerastuner.tuners import RandomSearch, Hyperband, BayesianOptimization
import numpy as np

## Подготовка данных для обучения сети

In [3]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [4]:
x_train = x_train.reshape(60000, 784)
x_test = x_test.reshape(10000, 784)
x_train = x_train / 255 
x_test = x_test / 255 
y_train = utils.to_categorical(y_train, 10)
y_test = utils.to_categorical(y_test, 10)

## Задаем функцию создания нейронной сети

In [21]:
def build_model(hp):
    model = Sequential()
    activation_choice = hp.Choice('activation', values=['relu', 'sigmoid', 'tanh', 'elu', 'selu'])    
    model.add(Dense(units=hp.Int('units_input',    # Полносвязный слой с разным количеством нейронов
                                   min_value=512,    # минимальное количество нейронов - 128
                                   max_value=1024,   # максимальное количество - 1024
                                   step=32),
                    input_dim=784,
                    activation=activation_choice))
    model.add(Dense(units=hp.Int('units_hidden',        
                                   min_value=128,   
                                   max_value=600,   
                                   step=32),
                    activation=activation_choice))   
    model.add(Dense(10, activation='softmax'))
    model.compile(
        optimizer=hp.Choice('optimizer', values=['adam','rmsprop','SGD']),
        loss='categorical_crossentropy',
        metrics=['accuracy'])
    return model

## Создаем tuner

Доступные типы тюнеров: 
- RandomSearch - случайный поиск.
- Hyperband - алгоритм оптимизации на основе многорукого бандита, Li, Lisha, and Kevin Jamieson. ["Hyperband: A Novel Bandit-Based Approach to Hyperparameter Optimization."Journal of Machine Learning Research 18 (2018): 1-52](http://jmlr.org/papers/v18/16-558.html).
- BayesianOptimization - [байесовская оптимизация](https://en.wikipedia.org/wiki/Bayesian_optimization).

In [22]:
tuner = RandomSearch(
    build_model,                 # функция создания модели
    objective='val_accuracy',    # метрика, которую нужно оптимизировать - 
                                 # доля правильных ответов на проверочном наборе данных
    max_trials=80,               # максимальное количество запусков обучения 
    directory='test_directory'   # каталог, куда сохраняются обученные сети  
    )

INFO:tensorflow:Reloading Oracle from existing project test_directory/untitled_project/oracle.json
INFO:tensorflow:Reloading Tuner from test_directory/untitled_project/tuner0.json


## Запускаем подбор гиперпараметров

Пространство поиска

In [13]:
tuner.search_space_summary()

Подбор гиперпараметров

In [14]:
tuner.search(x_train,                  # Данные для обучения
             y_train,                  # Правильные ответы
             batch_size=256,           # Размер мини-выборки
             epochs=20,                # Количество эпох обучения 
             validation_split=0.2,     # Часть данных, которая будет использоваться для проверки
             )

Epoch 1/20
188/188 [==============================] - 2s 10ms/step - loss: 0.8300 - accuracy: 0.7008 - val_loss: 0.5428 - val_accuracy: 0.8006
Epoch 2/20
188/188 [==============================] - 2s 9ms/step - loss: 0.5012 - accuracy: 0.8137 - val_loss: 0.4520 - val_accuracy: 0.8381
Epoch 3/20
188/188 [==============================] - 2s 9ms/step - loss: 0.4391 - accuracy: 0.8371 - val_loss: 0.4597 - val_accuracy: 0.8307
Epoch 4/20
188/188 [==============================] - 2s 9ms/step - loss: 0.4074 - accuracy: 0.8496 - val_loss: 0.4751 - val_accuracy: 0.8328
Epoch 5/20
188/188 [==============================] - 2s 9ms/step - loss: 0.3802 - accuracy: 0.8594 - val_loss: 0.4374 - val_accuracy: 0.8298
Epoch 6/20
188/188 [==============================] - 2s 9ms/step - loss: 0.3602 - accuracy: 0.8677 - val_loss: 0.3878 - val_accuracy: 0.8568
Epoch 7/20
188/188 [==============================] - 2s 9ms/step - loss: 0.3436 - accuracy: 0.8720 - val_loss: 0.3723 - val_accuracy: 0.8654
Epoch

INFO:tensorflow:Oracle triggered exit


## Выбираем лучшую модель

In [16]:
tuner.results_summary()

Получаем три лучших модели

In [19]:
models = tuner.get_best_models(num_models=8)

Оцениваем качество модели на тестовых данных

In [20]:
for model in models:
  model.summary()
  model.evaluate(x_test, y_test)
  print() 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 736)               577760    
_________________________________________________________________
dense_1 (Dense)              (None, 256)               188672    
_________________________________________________________________
dense_2 (Dense)              (None, 10)                2570      
Total params: 769,002
Trainable params: 769,002
Non-trainable params: 0
_________________________________________________________________
313/313 [==============================] - 1s 4ms/step - loss: 0.3647 - accuracy: 0.8909

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 672)               527520    
_________________________________________________________________
dense_